In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [3]:
model = load_model('model.h5')

# load encoders
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

with open('one_hot_encoder.pkl', 'rb') as f:
    onehot_encoder = pickle.load(f)

# load scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)


In [4]:
# example input data
data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,
}


In [9]:
df = pd.DataFrame([data])
# Encode categorical features
df['Gender'] = label_encoder.transform(df['Gender'])


In [6]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [10]:
geo = onehot_encoder.transform(df[['Geography']]).toarray()
df.drop('Geography', axis=1, inplace=True)
df = pd.concat([df, pd.DataFrame(geo, columns=onehot_encoder.get_feature_names_out(['Geography']))], axis=1, ignore_index=False)

In [11]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [12]:
# Scale numerical features
input_scaled = scaler.transform(df)

In [13]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [14]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step


array([[0.03309745]], dtype=float32)

In [17]:
prediction_prob = prediction[0][0]
if prediction_prob > 0.5:
    result = "Positive"
else:
    result = "Negative"
result

'Negative'